# Python implementation of L-Galaxies

This is a playground to test out the possibility of using `python` as an interface into L-Galaxies.

In [ ]:
# Imports of generic python routines

%load_ext autoreload
%autoreload 2

import astropy.constants as c
import astropy.units as u
import gc
import h5py
h5py.enable_ipython_completer()
import numpy as np
import sys

In [ ]:
# Parameters relating to the python code development.
# Parameters relating to the SAM will be set in the input yaml file.

# Location of code
C_DIR='code-C'
PYTHON_DIR='code-python'

# Development limiter
#n_GRAPH=np.inf
n_GRAPH=2

# Verbosity
VERBOSITY=4 # 0 - Major program steps only; 1/2 - Major/minor Counters; 3/4/5 - Debugging diags.

# Input files:
# List of all available options
FILE_OPTIONS_LIST='input/available_options.yml'
# List of runtime parameters
FILE_PARAMETERS='input/input.yml'

In [ ]:
# Imports of py-galaxies python routines
sys.path.insert(1,PYTHON_DIR)

# The parameter class, used to store run-time parameters
from parameters import C_parameters

# The graph class, used to store graphs for processing
from graphs import C_graph

# The halo class, used to store halo properties
from halos import C_halo

# The halo_output class and methods used to output halos
from halos import C_halo_output

# The subhalo class, used to store subhalo properties
from subhalos import C_subhalo

## Functions

Most of the work to be done in external routines, probably to be coded in C for efficiency.

Here we just include the high-level driver routines.

In [ ]:
# These routines will eventually be moved to halos.py.
# For now they are here to help with code development.

def F_process_halos(halos_this_snap,graph,parameters):
    """
    This is the controlling routine for halo processing.
    """
    for halo in halos_this_snap:
        if parameters.verbosity>=4: print('Processing halo ',halo.halo_ID)
        if halo.b_done==True:
            raise RuntimeError('halo '+str(halo.halo_ID)+' in graph '+str(halo.graph_ID)+' already processed.')
        F_set_baryon_fraction(halo,parameters)
        if parameters.b_HOD==True:
            F_set_stellar_fraction(halo,graph,parameters)
        halo.b_done=True
    
def F_update_halos(halos_last_snap,halos_this_snap,subhalos_last_snap,subhalos_this_snap,graph,parameters):
    """
    Propagate properties from progenitor halos to descendants.
    Done as a push rather than a pull because sharing determined by progenitor.
    First loop to push halo / subhalo properties; 
    then structured galaxy array needs to be generated in each halo;
    then second push of galaxy properties.
    """
    """
    Note that there are two offsets here:
    * Entries in the graph lookup tables are labelled by _start
    * Entries in the halo/subhalo instances lists are offset by (sub)halo_offset: labelled _snap
    """
    halo_offset=halos_this_snap[0].halo_ID
    if subhalos_this_snap != None: subhalo_offset=subhalos_this_snap[0].subhalo_ID
    if halos_last_snap != None: halo_offset_last=halos_last_snap[0].halo_ID
    if subhalos_last_snap != None: subhalo_offset_last=subhalos_last_snap[0].subhalo_ID
    for halo in halos_last_snap:
        # First determine what fraction to give to each descendant
        desc_start=halo.desc_start
        desc_end=desc_start+halo.n_desc
        if (halo.n_desc==0): 
            print('No descendants for halo:',halo,flush=True)
            # For now just skip this halo; might want in future to log these occurances
            continue
        fractions=graph.desc_contribution[desc_start:desc_end]/ \
            np.sum(graph.desc_contribution[desc_start:desc_end])
        desc_main_snap=np.argmax(fractions) # The descendant that gets most of our mass
        halo.desc_main_snap=desc_main_snap
        halos_this_snap[desc_main_snap].n_orphan+=halo.n_orphan # All orphans galaxies go to main descendant
        # Now loop over descendants transferring properties to them:
        for i_desc in range(desc_start,desc_end):
            desc_halo_ID=graph.desc_IDs[i_desc]
            desc_halo=halos_this_snap[desc_halo_ID-halo_offset]
            assert desc_halo_ID == desc_halo.halo_ID
            if parameters.verbosity>=5: print('Processing descendant',desc_halo_ID)
            desc_halo.mass_from_progenitors+=fractions[i_desc-desc_start]*halo.mass
            if parameters.b_HOD==True:
                desc_halo.mass_stars_from_progenitors+=fractions[i_desc-desc_start]*halo.mass_stars
                desc_halo.mass_stars+=fractions[i_desc-desc_start]*halo.mass_stars # Could be set later
    # Now loop over the subhalos
    if subhalos_last_snap != None:
        for subhalo in subhalos_last_snap:
            sub_desc_start=subhalo.desc_start
            sub_desc_end=sub_desc_start+subhalo.n_desc
            host_snap=subhalo.host-halo_offset_last
            print(subhalo.host,halo_offset_last,host_snap,flush=True)
            desc_main_snap=halos_last_snap[host_snap].desc_main_snap
            subhalo.desc_host_snap=desc_main_snap
            if sub_desc_end==sub_desc_start:
                # If no descendant subhalo components get given to the (main descendant of) the host halo
                halos_this_snap[desc_main_snap].n_orphan+=subhalo.n_gal
            else:
                # Otherwise the main subhalo descendant gets all the galaxies
                fractions=graph.sub_desc_contribution[sub_desc_start:sub_desc_end]/ \
                    np.sum(graph.sub_desc_contribution[sub_desc_start:desc_end])
                sub_desc_main_snap=np.argmax(fractions)
                subhalo.desc_main_snap=sub_desc_main_snap
                subhalos_this_snap[sub_desc_main_snap].n_gal+=subhalo.n_gal
                subhalos_this_snap[sub_desc_main_snap].ICM_mass+=subhalo.ICM_mass
                subhalos_this_snap[sub_desc_main_snap].hot_gas_mass+=subhalo.hot_gas_mass
                # Now loop over descendants transferring properties to them
                # Only required if we decide that subhalos can split
                # for i_desc in range(sub_desc_start,sub_desc_end):
                #     desc_subhalo_ID=graph.sub_desc_IDs[i_desc]
                #     desc_subhalo=subhalos_this_snap[desc_subhalo_ID-subhalo_ID_offset]
                #     assert desc_subhalo_ID == desc_subhalo.subhalo_ID
                #     sub_desc_halo.<quantity>+=fractions[i_desc-desc_start]*subhalo.<quantity>
    # Now count the total number of galaxies and generate the galaxy array.
    # Note that done as a loop over subhalos within halos so as to keep all galaxies in
    # a halo closely associated in the array.
    n_gal=0
    for halo in halos_this_snap:
        n_gal_start=n_gal
        if halo.n_sub>0:
            for subhalo in subhalos_this_snap[halo.sub_start_snap:halo.sub_end_snap]:
                subhalo.init_gal(n_gal)
        halo.init_gal(n_gal_start,n_gal)
    # Create new galaxy array
    galaxies_this_snap=np.empty(n_gal,dtype=parameters.dtype_gal) 
    # Second loop to pass on galaxy properties.
    if galaxies_last_snap == None: return
    for halo in halos_last_snap:
        n_orphan=halo.n_orphan
        if n_orphan > 0:
            # match up orphans
            desc_halo=halos_this_snap[halo.desc_main_snap]
            gal_last_start=halo.orphan_start
            gal_last_end=gal_last_start+n_orphan
            gal_this_start=desc_halo.add_gal(n_orphan)
            gal_this_end=gal_this_start+n_orphan
            # Copy over all properties
            galaxies_this_snap[gal_this_start:gal_this_end]=galaxies_last_snap[gal_last_start:gal_last_end]
            # Update the tracking pointers
            galaxies_this_snap[gal_this_start:gal_this_end]['halo_graph']=desc_halo.halo_ID
            galaxies_this_snap[gal_this_start:gal_this_end]['halo_snap']=desc_halo.halo_ID-halo_offset
            galaxies_this_snap[gal_this_start:gal_this_end]['sub_graph']=parameters.NO_DATA_INT
            galaxies_this_snap[gal_this_start:gal_this_end]['sub_graph']=parameters.NO_DATA_INT
            galaxies_this_snap[gal_this_start:gal_this_end]['first_prog_snap']=range(gal_this_start,gal_this_end)
            galaxies_this_snap[gal_this_start:gal_this_end]['next_prog_snap']=parameters.NO_DATA_INT
    if subhalos_last_snap != None:
        for subhalo in subhalos_last_snap:
            n_gal=subhalo.n_gal
            sub_desc_start=subhalo.desc_start
            sub_desc_end=sub_desc_start+subhalo.n_desc
            if sub_desc_end==sub_desc_start:
                # If no descendant galaxies become orphans of (the main descendant of) the host halo
                desc_halo=halos_this_snap[subhalo.desc_host_snap]
                gal_last_start=subhalo.gal_start
                gal_last_end=gal_last_start+n_gal
                gal_this_start=desc_halo.add_gal(n_gal)
                gal_this_end=gal_this_start+n_gal
                # Copy over all properties
                galaxies_this_snap[gal_this_start:gal_this_end]=galaxies_last_snap[gal_last_start:gal_last_end]
                # Update the tracking pointers
                galaxies_this_snap[gal_this_start:gal_this_end]['halo_graph']=desc_halo.halo_ID
                galaxies_this_snap[gal_this_start:gal_this_end]['halo_snap']=desc_halo.halo_ID-halo_offset
                galaxies_this_snap[gal_this_start:gal_this_end]['sub_graph']=parameters.NO_DATA_INT
                galaxies_this_snap[gal_this_start:gal_this_end]['sub_graph']=parameters.NO_DATA_INT
                galaxies_this_snap[gal_this_start:gal_this_end]['first_prog_snap']=range(gal_this_start,gal_this_end)
                galaxies_this_snap[gal_this_start:gal_this_end]['next_prog_snap']=parameters.NO_DATA_INT
            else:
                # Otherwise the main subhalo descendant gets all the galaxies
                desc_subhalo=subhalos_this_snap[subhalo.desc_main_snap]
                desc_halo=halos_this_snap[desc_subhalo.host-subhalo_offset]
                gal_last_start=halo.gal_start
                gal_last_end=gal_last_start+n_gal
                gal_this_start=desc_subhalo.add_gal(n_gal)
                gal_this_end=gal_this_start+n_gal
                # Copy over all properties
                galaxies_this_snap[gal_this_start:gal_this_end]=galaxies_last_snap[gal_last_start:gal_last_end]
                # Update the tracking pointers
                galaxies_this_snap[gal_this_start:gal_this_end]['halo_graph']=desc_halo.halo_ID
                galaxies_this_snap[gal_this_start:gal_this_end]['halo_snap']=desc_halo.halo_ID-halo_offset
                galaxies_this_snap[gal_this_start:gal_this_end]['sub_graph']=desc_subhalo.halo_ID
                galaxies_this_snap[gal_this_start:gal_this_end]['sub_graph']=desc_subhalo.halo_ID-subhalo_offset
                galaxies_this_snap[gal_this_start:gal_this_end]['first_prog_snap']=range(gal_this_start,gal_this_end)
                galaxies_this_snap[gal_this_start:gal_this_end]['next_prog_snap']=parameters.NO_DATA_INT


def F_update_parameters(graph_file,parameters):
    for key, value in graph_file['Header'].attrs.items():
        if parameters.b_display_parameters: print(key,value)
        exec('parameters.'+key+'=value')
    parameters.n_graph=len(graph_file['graph_lengths'])
    # Put code in here to either copy table of snapshot redshifts/times from graph_file,
    # Or calculate them if that does not exist.
    # Currently read from disk:
    parameters.snap_table=np.loadtxt(parameters.snap_input_file,usecols=[0,2,4],
        dtype=[('snap_ID',np.int32),('redshift',np.float32),('time_in_years',np.float32)])

# These routines will be replaced by interfaces to existing L-Galaxies routines, 
# written in C and located in code-C/

def F_set_baryon_fraction(halo,parameters):
    halo.mass_baryon=parameters.baryon_fraction*max(halo.mass,halo.mass_from_progenitors)
    
# This one is a for development testing
# It's full of magic numbers; but it's only a fudge, so not putting them in parameter file
def F_set_stellar_fraction(halo,graph,parameters):
    if halo.mass>1e12:
        halo.star_formation_rate=0.
    else:
        max_mass_stars=0.1*halo.mass_baryon
        halo.star_formation_rate=max(0.,(max_mass_stars-halo.mass_stars)/3e9)
    dt=parameters.snap_table['time_in_years'][halo.snap_ID]- \
        parameters.snap_table['time_in_years'][halo.snap_ID-1]
    halo.mass_stars+=halo.star_formation_rate*dt

## Main routine

### Initialisation

In [ ]:
# Read in all the parameters of the run from the yaml and graph input files.

# Read in parameters from yaml input files
parameters=C_parameters(FILE_PARAMETERS,FILE_OPTIONS_LIST)

# Open graph input file
graph_file=h5py.File(parameters.graph_input_file,'r')

# Update parameters with information from graph_file
F_update_parameters(graph_file,parameters)

# Create halo output buffer
halo_output=C_halo_output(parameters)

### Loop over graphs, snapshots, halos, implementing the SAM

In [ ]:
# Loop over graphs
for i_graph in range(min(parameters.n_graph,n_GRAPH)):
    if VERBOSITY >= 2: print('Processing graph',i_graph,flush=True)
    graph = C_graph(i_graph,graph_file,parameters)
    
    # Loop over snapshots
    halos_last_snap = None
    for i_snap in graph.snap_ID:
        if i_snap == parameters.NO_DATA_INT: 
            assert halos_last_snap == None
            continue
        if VERBOSITY >= 3: print('Processing snapshot',i_snap,flush=True)
            
        # Initialise halo and subhalo properties.
        # This returns a list of halo and subhalo instances
        halos_this_snap = [C_halo(i_graph,i_snap,i_halo,graph,parameters) for i_halo in 
                         graph.halo_start[i_snap]+range(graph.n_halo_snap[i_snap])]
        subhalos_this_snap = None
        if graph.n_sub > 0:
            if graph.n_sub_snap[i_snap] > 0:
                subhalos_this_snap = [C_subhalo(i_graph,i_snap,i_sub,graph,parameters) 
                                     for i_sub in graph.sub_start[i_snap]+range(graph.n_sub_snap[i_snap])]

        # We don't know how many galaxies we need at this point, so an equivalent array for galaxies
        # will be set during the following update_halos step.  For now set dummy variable to be overwritten
        galaxies_this_snap = None
        
        # Propagate information from progenitors to this generation
        # Done as a push rather than a pull because sharing determined by progenitor
        if halos_last_snap != None: 
            F_update_halos(halos_last_snap,halos_this_snap,subhalos_last_snap,subhalos_this_snap,graph,parameters)
            del halos_last_snap
            del subhalos_last_snap
            del galaxies_last_snap
            #gc.collect() # garbage collection -- safe but very slow.
        # Process the halos
        F_process_halos(halos_this_snap,graph,parameters)
            
        # Once all halos have been done, output results
        # This could instead be done on a halo-by-halo basis in F_process_halos
        halo_output.append(halos_this_snap,parameters)
        #subhalo_output.append(subhalos_this_snap,parameters)
        #galaxy_ouput(galaxies_this_snap,parameters)
            
        # Tidy up
        try:
            halos_last_snap=halos_this_snap
            subhalos_last_snap=subhalos_this_snap
            galaxies_last_snap=galaxies_this_snap
        except:
            pass

    # Free up memory
    try:
        del halos_last_snap
        del halos_this_snap
        del subhalos_last_snap
        del subhalos_this_snap
        del galaxies_last_snap
        del galaxies_this_snap
    except:
        pass


###  Tidy up and exit

In [ ]:
# Flush buffers, close files and exit
halo_output.close()
graph_file.close()